## Tokenization

In [ ]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    verdict = f.read()

import re
alltext = re.findall(r'\w+|[,.!?"\']|--',verdict)


alltext = sorted(set(alltext)) #sort the text

print(alltext)

vocab = {key:value for key,value in enumerate(alltext)} ## vocab
# for i,item in enumerate(vocab.items()):
#     if(i<50):
#         print(i,item)



In [ ]:
class SimpleClassTokenizer:
    def __init__(self,vocab):
        self.str_to_int = {s:i for i,s in vocab.items()}
        self.int_to_str = vocab
    
    def encode(self,text):
        preprocessed = re.findall(r'\w+|[,.!?"\']|--',text)
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
     
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.!?"()\'])', r'\1', text)
        return text


In [ ]:
tokenizer = SimpleClassTokenizer(vocab)

text1 = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text1)
print(ids)
tokenizer.decode(ids)


In [ ]:
# print(alltext)
alltext2 = alltext
alltext2.extend(['<|endoftext|>','<|unk|>']) ## Add endline
print(len(alltext2))
vocab = {i:s for i,s in enumerate(alltext2)}
print(len(vocab))


In [ ]:
alltext.clear()

In [ ]:
class SimpleClassTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = {s:i for i,s in vocab.items()}
        self.int_to_str = vocab

    def encoding(self,text):
        preprocessed = re.findall(r'\w+|--|[,.:;\-()?!"\']',text)
        preprocessed = [item if item in self.str_to_int else '<|endoftext|>' for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decoding(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+(--|[,.:;\-()?!"\'])',r'\1',text)
        return text


In [ ]:
tokenizer = SimpleClassTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text0 = " <|endoftext|> ".join((text1, text2))

print(text0)
ids = tokenizer.encoding(text0)
print(ids)
print(tokenizer.decoding(ids))

## Byte Pair Encoding

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
token_ids = tokenizer.encode(verdict,allowed_special={"<|endoftext|>"})
text = tokenizer.decode(token_ids)
print(len(token_ids))
print(len(text))
print(text[:50])

## Create Input and Output Pairs

In [5]:
with open('the-verdict.txt',encoding='utf-8') as f:
    rawdata = f.read()
print(len(rawdata))

import tiktoken
import torch
from torch.utils.data import Dataset,DataLoader

class SimpleClassDataset(Dataset):
    def __init__(self,txt,tokenizer,stride, maxlength):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt,allowed_special={'<|endoftext|>'})

        for i in range(0,len(token_ids)-maxlength,stride):
            input_chunk = token_ids[i:i+maxlength]
            target_chunk = token_ids[i+1:i+maxlength+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index],self.target_ids[index]
    
def create_dataloader(text,stride=128,batch_size=4,max_length=256,shuffle=True,drop_last = True,num_worker=0):

    tokenizer = tiktoken.get_encoding('gpt2')
    
    dataset = SimpleClassDataset(text,tokenizer,stride,max_length)

    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_worker
    )
    
    return dataloader


dataloader = create_dataloader(text=rawdata,batch_size=6,stride=4,max_length=4,shuffle=False,drop_last=True)

data_iter = iter(dataloader)
input,target = next(data_iter)
print(input)
print(target)
print(len(dataloader))



20479
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049]])
214


## Token Embedding

In [ ]:
torch.manual_seed(123)
embedding_layer= torch.nn.Embedding(50257,256)
print(input.shape)
token_embedding = embedding_layer(input)
print(token_embedding.shape)

torch.Size([4, 4])
torch.Size([4, 4, 256])


## Positional Embedding

In [ ]:
max_length = 4
dimensions = 256
pos_embedding_layer = torch.nn.Embedding(max_length,dimensions)
pos_embedding = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding)

tensor([[ 0.5423, -0.1224, -1.4150,  ...,  0.2515, -2.3067,  0.8155],
        [-0.3973, -1.2575, -1.9800,  ..., -0.1207,  0.3075, -0.6422],
        [ 0.1840,  1.1128,  1.0052,  ...,  0.2081,  0.5531, -1.1619],
        [ 1.4155,  0.6599,  0.3760,  ...,  0.7034, -0.6108,  0.1080]],
       grad_fn=<EmbeddingBackward0>)


## Input Embedding

In [15]:
input_embeddings = token_embedding + pos_embedding
print(input_embeddings.shape)

torch.Size([4, 4, 256])


## Simple Self-Attention

In [1]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
attention_score = inputs @ inputs.T
attention_weights = torch.softmax(attention_score, dim = -1)
print(attention_weights)
print(attention_weights.sum(dim=-1))

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [7]:
all_context_vector = attention_weights @ inputs
print(all_context_vector)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## Self-Attention

In [ ]:
import torch 
import torch.nn as nn
class SelfAttention(nn.Module):
    def __init__(self, d_in,d_out,qkv_bias):
        super().__init__()
        self.w_query = nn.Linear(d_in,d_out,qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,qkv_bias)

    
    def forward(self,x):
        query = self.w_query(x)
        key = self.W_key(x)
        value = self.W_value(x)
        attention_score = query @ key.T
        attention_weights = torch.softmax(attention_score/key.shape[-1]**0.5,dim=-1)
        context_vector = attention_weights @ value

        return context_vector
    

## Casual Attention

Text<br>
↓<br>
Tokenization --- (length)<br>
↓<br>
DataLoader -----(batch_size, contextlength) (4,6)<br>
↓<br>
Token Embedding ---(batch_size,context_length,dimensions) (4,6,512) + <br>
Positional Embedding --(batch_size,context_length,dimensions) (4,6,512)<br>
↓<br>
Query,Key,Value weights---(dimensions,attention_head)/(d_in,d_out) +<br>
Query,Key,Value ---(batch,context_length,d_out)<br>
Attention_score ---(batch,context_length,context_length)<br>
context_vector ---(batch,token,d_out)<br>
Self-Attention---(batch,tokens,d_out) <br>



In [ ]:
import torch
import torch.nn as nn
class CasualAttention(nn.Module):
    def __init__(self, d_in,d_out,context_length,qkv_bias,dropout):
        super().__init__()
        self.d_out = d_out
        self.w_query = nn.Linear(d_in,d_out,qkv_bias) ## d_in should be equal to the dimensions of the embedding
        self.w_key = nn.Linear(d_in,d_out,qkv_bias)
        self.w_value = nn.Linear(d_in,d_out,qkv_bias)
        self.drop = nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    def forward(self,x):
        b, num_tokens, d_in = x.shape
        query = self.w_query(x)
        key = self.w_key(x)
        value = self.w_value(x)

        attention_score = query @ key.transpose(-1,-2)
        attention_score.masked_fill_(self.mask.bool()[:num_tokens,:num_tokens],-torch.inf) ## PyTorch broadcasting rules
        attention_weights = torch.softmax(attention_score/key.shape[-1]**0.5,dim=-1)
        attention_weights = self.drop(attention_weights) # New

        context_vec = attention_weights @ value
        return context_vec

    
        

## Multi Head Attention Wrapper Class

In [ ]:
class multiheadwrapper(nn.Module):
    ## head_dim = d_out/num_heads
    def __init__(self,d_in,d_out,context_length,num_heads,dropout,qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CasualAttention(d_in,d_out,context_length,qkv_bias,dropout)
             for _ in range(num_heads)]   
        )

    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim =-1) ## concat by increasing the dimensions of the token

In [3]:
import torch
import torch.nn as nn
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in,d_out,context_length,num_heads,dropout,qkv_bias= False):
        super().__init__()
        assert(d_out%num_heads==0),"d_out should be divisible by the number of heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.w_query = nn.Linear(d_in,d_out,qkv_bias)
        self.w_key = nn.Linear(d_in,d_out,qkv_bias)
        self.w_value = nn.Linear(d_in,d_out,qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    def forward(self,x):

        b,num_token,d_in = x.shape

        query = self.w_query(x)
        key = self.w_key(x)
        value = self.w_value(x)

        query.view(b,num_token,self.num_heads,self.head_dim)
        key.view(b,num_token,self.num_heads,self.head_dim)
        value.view(b,num_token,self.num_heads,self.head_dim)

        query = query.transpose(1,2)
        key = key.transpose(1,2)
        value = value.transpose(1,2)

        attention_score = query @ key.transpose(2,3)
        attention_score.masked_fill_(self.mask.bool()[:num_token,:num_token],-torch.inf)
        attention_weights = torch.softmax(attention_score/key.shape[-1]**0.5,dim=-1) 
        attention_weights = self.drop(attention_weights)
        context_vector = (attention_weights @ value).transpose(1,2)
        context_vector = context_vector.contiguous().view(b,num_token,self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vector
